<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/riva_asr_asr-python-advanced-customize-vocabulary-and-lexicon/nvidia_logo.png" style="width: 90px; float: right;">

# How to Customize Riva ASR Vocabulary and Pronunciation with Lexicon Mapping

This notebook walks you through the process of customizing Riva ASR NIM vocabulary and lexicon, in order to improve Riva vocabulary coverage and recognition of difficult words, such as acronyms.

## Overview

The Flashlight decoder, deployed by default in Riva Parakeet-CTC NIM, is a lexicon-based decoder and only emits words that are present in the provided lexicon file. That means, uncommon and new words, such as domain specific terminologies, that are not present in the lexicon file, will have no chance of being generated.

On the other hand, the greedy decoder (available as an option during the `riva-build` process with the flag `--decoder_type=greedy`) is not lexicon-based and hence can virtually produce any word or character sequence.

### Prerequisite

This notebook assumes that the user is familiar with manually deploying a Riva ASR NIM using the Riva ServiceMaker tool, `riva-build` and `riva-deploy` commands. See Riva NIM [documentation](https://docs.nvidia.com/nim/riva/asr/latest/custom-deployment.html#deploying-custom-models-as-nim).

### Terminologies
- **Vocabulary file**: The vocabulary file is a flat text file containing a list of vocabulary words, each on its own line. For example:
```
the
i
to
and
a
you
of
that
...
```

This file is used by the `riva-build` process to generate the lexicon file. 

- **Lexicon file**: The lexicon file is a flat text file that contains the mapping of each vocabulary word to its tokenized form, e.g, sentencepiece tokens, separated by a `tab`. Below is an example:

```
with    ▁with
not     ▁not
this    ▁this
just    ▁just
my      ▁my
as      ▁as
don't   ▁don ' t
```

*Note: Ultimately, the Riva decoder makes use only of the lexicon file directly at run time (but not the vocabulary file).*

Riva ServiceMaker automatically tokenizes the words in the vocabulary file to generate the lexicon file. It uses the correct tokenizer model that is packaged together with the acoustic model in the `.riva` file. By default, Riva generates 1 tokenized form for each word in the vocabulary file. 

## What can be customized?

Both the vocabulary and the lexicon files can be customized.

- Extending the vocabulary enriches the Riva default vocabulary, providing additional coverage for out-of-vocabulary words, terminologies, and abbreviations.

- Customizing the lexicon file can further enrich the Riva knowledge base by providing one or more explicit pronunciations, in the form of tokenized sequences.

## Extending the vocabulary

Extending the vocabulary must be done at Riva **build** time.

When building a Riva ASR NIM pipeline, pass the [extended vocabulary file](#modify_vocab) to the `--decoding_vocab=<vocabulary_file>` parameter of the build command. For example, the build command for the Parakeet model:

```
    riva-build speech_recognition \
   <rmir_filename>:<key> <riva_filename>:<key> \
   --name=parakeet-english-asr-streaming \
   --ms_per_timestep=80 \
   --decoding_language_model_binary=<lm_binary> \
   --decoding_vocab=<vocabulary_file> \
   --language_code=en-US \
   <other_parameters>...
```

Refer to Riva NIM [documentation](https://docs.nvidia.com/nim/riva/asr/latest/pipeline-configuration.html) for build commands for supported models.




<a id='modify_vocab'></a>
### How to modify vocabulary file

You can either provide your own vocabulary file, or extend Riva's default vocabulary file.

- BYO vocabulary file: provide a flat text file containing a list of vocabulary words, each on its own line. Note that this file must not only contain a small list of "difficult words", but must contains all the words that you want the ASR pipeline to be able to generate, that is, including all common words.

- Modifying an existing one: This is the recommended approach. Out-of-the-box vocabulary files  for Riva supported languages can be found on NGC, for example, for English, the vocabulary file named `flashlight_decoder_vocab.txt` can be found at this [link](https://catalog.ngc.nvidia.com/orgs/nim/teams/nvidia/models/parakeet-1-1b-ctc-en-us_finetune).
   
You can extend this default vocabulary file with the words of interest.

Once modified, you'll have to deploy the Riva ASR NIM with `riva-build` while passing the flag `--decoding_vocab=<modified_vocabulary_file>`.

## Customizing pronunciation with lexicon mapping

The lexicon file that is used by the Flashlight decoder can be found in the Riva models directory inside the NIM container. If you have deployed Parakeet 1.1B CTC en-US NIM, following command can be used to get the lexicon file.
```bash
        # Copy the lexicon file from the docker to the current directory
        docker cp parakeet-1-1b-ctc-en-us:/data/models/parakeet-1.1b-en-US-asr-streaming-asr-bls-ensemble/1/lexicon.txt /dest
```      

### How to modify the lexicon file

Modify the lexicon file to add the sentencepiece tokenizations for the words of interest. For example, one could add:
```
manu ▁ma n u
manu ▁man n n ew
manu ▁man n ew
```
which are 3 different pronunciations/tokenizations of the word `manu`.  If the acoustic model predicts those tokens, they will be decoded as `manu`.

Once this is done, regenerate the model repository using that new decoding lexicon tokenization by passing `--decoding_lexicon=modified_lexicon.txt` to `riva-build` instead of `--decoding_vocab=decoding_vocab.txt`.

### How to generate the correct tokenized form

When modifying the lexicon file, ensure that:

- The new lines follow the indentation/space pattern like the rest of the file and that the tokens used are part of the tokenizer model. 

- The tokens are valid tokens as determined by the tokenizer model (packaged with the Riva acoustic model).

The latter ensures that you use only tokens that the acoustic model has been trained on. To do this, you’ll need the tokenizer model and the `sentencepiece` Python package (`pip install sentencepiece`). You can get the tokenizer model for the deployed pipeline from the model repository `...-asr-bls-ensemble` directory for your model. It will be named `<hash>_tokenizer.model`. For example:

`/data/models/parakeet-1.1b-en-US-asr-streaming-asr-bls-ensemble/1/ef7c2379c9624a688ad233ab2d455e35_tokenizer.model`

When using a docker volume to store Riva assets (by default), you can copy the tokenizer model to the local directory with a command such as:

```bash
        # Copy the tokenizer model file from the docker image to the current directory
        docker cp parakeet-1-1b-ctc-en-us:/data/models/parakeet-1.1b-en-US-asr-streaming-asr-bls-ensemble/1/ef7c2379c9624a688ad233ab2d455e35_tokenizer.model /dest
```  

In [ ]:
!pip install sentencepiece

You can then generate new lexicon entries, for example:
    

In [ ]:
TOKEN="BRAF"
PRONUNCIATION="b raf"

import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='tokenizer.model')
for n in range(5):
    print(TOKEN + '\t' + ' '.join(s.encode(PRONUNCIATION, out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1)))

Note: `TOKEN` represents the desired written form of the word, while `PRONUNCIATION` is what the word should sound like.

Other examples:

In [ ]:
TOKEN="WhatsApp"
PRONUNCIATION="what's app"

import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='tokenizer.model')
for n in range(5):
    print(TOKEN + '\t' + ' '.join(s.encode(PRONUNCIATION, out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1)))

In [ ]:
TOKEN="Cya"
PRONUNCIATION="See ya"

import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='tokenizer.model')
for n in range(5):
    print(TOKEN + '\t' + ' '.join(s.encode(PRONUNCIATION, out_type=str, enable_sampling=True, alpha=0.1, nbest_size=-1)))

## Go deeper into Riva capabilities


### Additional Riva Tutorials

Checkout more Riva tutorials [here](https://github.com/nvidia-riva/tutorials) to understand how to use some of the advanced features of Riva ASR, including customizing ASR for your specific needs.


### Additional Resources

For more information about each of the Riva APIs and their functionalities, refer to the [documentation](https://docs.nvidia.com/nim/riva/asr/latest/protos.html).